Author: Amy Weng

This file is for running named entity recognition (NER) using spaCy on individual Restoration-era texts that mention BOTH monopoly and East India Company. 

In [159]:
import os
import re
import pandas as pd
import string
import spacy
directory = r'C:\Users\amycw\Desktop\ecbc research'
eicFolder = directory + "\\"+'eic_monopoly_eebo'

In [9]:
# check for files that are bigger than 1000000 
for file in os.listdir(eicFolder):
    entries = pd.read_csv(eicFolder + "\\" + file)
    num_rows = entries.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = entries[i:(i+1)]
        text = data.iloc[0].text
        title = data.iloc[0].title
        if (len(text)>=1000000):
            print(title+"\n")
            print(len(text))

the history of england giving a true and impartial account of the most considerable transactions in church and state in peace and war during the reigns of all the kings and queens from the coming of julius caesar into britain with an account of all plots conspiracies insurrections and rebellions likewise a relation of the wonderful prodigies to the year together with a particular description of the rarities in the several counties of england and wales with exact maps of each county by john seller 

1185040


In [11]:
#running named entity recognition using spaCy on eicFolder
nerFile = open(r'C:\Users\amycw\Desktop\ecbc research\namedEntityRecognition.txt', "a")
ner = spacy.load('en_core_web_sm')
for fileName in os.listdir(eicFolder):
    readFile = pd.read_csv(eicFolder + "\\" + fileName)
    for i in range(len(readFile.index)):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        entities = ner(text)
        for ent in entities.ents:
            nerFile.write(ent.text+" "+ent.label_+"\n")
nerFile.close()

In [12]:
# alphabetize and sort entities by frequency

nerFile = open(r'C:\Users\amycw\Desktop\ecbc research\namedEntityRecognition.txt', "r")
SortedNER = open(r'C:\Users\amycw\Desktop\ecbc research\sortedNamedEntityRecognition.txt',"a")
SortedNER.write("format is (<name> <TYPE>, <frequency>)\n")
uniqueEnts = {} 
entities = []
indices = []
entities = nerFile.read().split("\n")
for ent in entities:
    if ent not in indices: 
        uniqueEnts[ent] = 1
        indices.append(ent)
    else: 
        uniqueEnts[ent] = uniqueEnts[ent] + 1 
alphabetizedEnts = {}
for ent in sorted(uniqueEnts): 
    alphabetizedEnts[ent]=uniqueEnts[ent] 
for pair in sorted (alphabetizedEnts.items(), key=lambda kv:(kv[1], kv[0]), reverse=True):
    SortedNER.write(str(pair)+"\n")
SortedNER.close()

In [10]:
#break the too big file into two smaller ones 

breakFile = r'C:\Users\amycw\Desktop\ecbc research\eic_monopoly_eebo\A5_P4_3070.csv'
entries= pd.read_csv(breakFile)
half = []
otherHalf = []
count = 0 
string = ""
other = ""
for i in range(len(entries.index)):
    d = entries[i:(i+1)]
    text = d.iloc[0].text
    words = text.split(" ")
    for w in words: 
        if (count <= (len(words)/2)):
            half.append(w)
            count+=1
        else:
            otherHalf.append(w)
for w in half:
    string = string + w + " "
for w in otherHalf: 
    other = other + w + " "

df1 = pd.DataFrame([(d.iloc[0].index, d.iloc[0].title,d.iloc[0].author,d.iloc[0].publisher,d.iloc[0].date,string)],
           columns=('index', 'title','author','publisher','date','text'))
df2 = pd.DataFrame([(d.iloc[0].index, d.iloc[0].title,d.iloc[0].author,d.iloc[0].publisher,d.iloc[0].date,other)],
           columns=('index', 'title','author','publisher','date','text'))
df1.to_csv(eicFolder + "\\" + 'A5_P4_3070_1.csv', index=False)
df2.to_csv(eicFolder + "\\" + 'A5_P4_3070_2.csv', index=False)